# Web Scraping 

#### General Setup

Notes: The code requires a stable internet connnection and for the computer screen to be on while all the cells are finished running. 

* See README doc for additional information about setup, in addition to a summary of this code notebook and the data columns.
https://docs.google.com/document/d/1M1hDcaXNH4CMZ1qvaVwoN1mmEF_xYIqouQNwlLTX7Cg/edit?usp=sharing

* This notebook can be run with 1 input: an excel file containing a sheet labeled "search_terms" which has in its first column a list of chemicals. This excel file can be imported in the cell labeled: "# Creating the Google search terms: final_search_terms is a list of the terms".

In [ ]:
# Import Cell 

import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from babel.numbers import get_currency_symbol, list_currencies
from babel.localedata import locale_identifiers
import requests
import re 
import time
import random 
import math

# 
chrome_options = Options()
chrome_options.add_argument('--disable-http2')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# Global Lists - to store data

list_search_term = []
list_parent_site = []
list_website = []
list_product_name = []
list_date = []

list_units = [] 
list_price = []

list_category_1_name = []
list_category_1 = []
list_category_2_name = []
list_category_2 = []


Site-specific functions for scraping plus helper functions. Information that can be collected is either labeled through a hashtag or is already labeled.

- Try except blocks are used to allow for data to be collected if it exists.

In [ ]:
# 1: FisherSci 

def fishersci(website_link, search_term):
    # Globally defined lists 
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2
    
    # Wait for the "reject all" button to be clickable and click it.
    try:
    
        driver.get(website_link) 
        reject_all_button = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.XPATH, '//button[text()="Reject All"]'))
        )
        reject_all_button.click()
    except Exception as e:
        pass    
    
    # Filtering for links that include fishersci.com/shop. Links that don't include fishersci.com/shop don't contain useful data.
    if 'fishersci.com/shop' in website_link: 

        product_name = driver.find_element(By.TAG_NAME, "h1").text
        
        # Pressing all buttons, which takes us to new pages. 
        button_string = 'attributeButton_Quantity_0'
        i = 0
        while True: 
            try:
                
                button_string = button_string[:-1]
                button_string += str(i)
                button = driver.find_element(By.ID, button_string)
                button.click()

                time.sleep(3)

                new_url = driver.current_url   # The new url 

                # Grabbing data - Quantity
                try:
                    quantity = driver.find_element(By.ID, button_string).text 
                except Exception as e: 
                    quantity = "Not Found"
            
                # Packaging - assuming 1 to 1 (quantity - packaging)
                try:
                    packaging = driver.find_element(By.XPATH, "//div[contains(@class, 'attributeButton') and contains(@class, 'Packaging') and  contains(@class, 'selected')]").text
                except Exception as e:
                    packaging = None
        
                try:
                    # Prices - would like to grab both price and units
                    prices = driver.find_elements(By.XPATH, "//b[@data-stockromm-sku]")
                    units = driver.find_elements(By.XPATH, "//span[@class='qa_single_display_unit']/span")
                    prices_and_units = tuple(zip(prices, units))
                    price_text = []
                    units_text = []
                    for j, k in prices_and_units:
                        list_search_term = np.append(list_search_term, search_term)
                        list_parent_site = np.append(list_parent_site, "FisherScientific") # Parent Site
                        list_product_name = np.append(list_product_name, product_name) # Product Name
                        list_website = np.append(list_website, new_url) # Website
                        price_text = np.append(price_text, j.text)  # Price
                        units_text = np.append(units_text, k.text) # Units
                        list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S")) # Date - Finished scraping the page

                        list_category_1_name = np.append(list_category_1_name, "Quantity")
                        list_category_1 = np.append(list_category_1, quantity)
                        list_category_2_name = np.append(list_category_2_name, "Packaging")
                        list_category_2 = np.append(list_category_2, packaging)
              
                    list_price = np.append(list_price, price_text)
                    list_units = np.append(list_units, units_text)  
                    i += 1
                except Exception as e:
                    continue
        
            except Exception as e:
                break   

In [ ]:
# 2: Msesupplies

def msesupplies(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date,list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2
    if 'msesupplies.com/products' in website_link:

        try: 
            driver.get(website_link)
            time.sleep(1)
            product_name = driver.find_element(By.TAG_NAME, "h1").text

            # Grabbing table, option selectors. Msesupplies sites include a table or a dropdown menu (option selector), or neither. 

            tbl = driver.find_elements(By.XPATH, "//table[@class='shappify_qb_grid']/tbody/tr/td")
            select_element = driver.find_element(By.XPATH, "//select[@class='single-option-selector']")
            select_object = Select(select_element)
            select_options = select_object.options
            
            # If a table is detected.
            if tbl:
                td_texts = [td.text for td in tbl]
                nonempty_texts = [a for a in td_texts if a.strip()] # Want to translate this into better form
                quantities = nonempty_texts[0::2] # one value per row in the table
                prices = nonempty_texts[1::2]

                quantities_prices = tuple(zip(quantities, prices))
                for q, p in quantities_prices:
                    list_search_term = np.append(list_search_term, search_term)
                    list_parent_site = np.append(list_parent_site, "msesupplies")
                    list_website = np.append(list_website, website_link)
                    list_product_name = np.append(list_product_name, product_name)
                    list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    # list_quantity = np.append(list_quantity, q)
                    # list_packaging = np.append(list_packaging, None)
                    list_units = np.append(list_units, 1)
                    list_price = np.append(list_price, p)

                    list_category_1_name = np.append(list_category_1_name, q)
                    list_category_1 = np.append(list_category_1, "Quantity")
                    list_category_2_name = np.append(list_category_2_name, None)
                    list_category_2 = np.append(list_category_2, None)

            # If a dropdown menu/option selector is detected.
            elif len(select_options) != 1:
                select_element = driver.find_element(By.XPATH, "//select[@class='single-option-selector']")
                
  
                for option in select_options:
                    select_object.select_by_visible_text(option.text)

                    # Now, extract price for each option. 
                    price = driver.find_element(By.XPATH, "//span[@id='productPrice']").text

                    list_search_term = np.append(list_search_term, search_term)
                    list_parent_site = np.append(list_parent_site, "msesupplies")
                    list_website = np.append(list_website, website_link)
                    list_product_name = np.append(list_product_name, product_name)
                    list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    # list_quantity = np.append(list_quantity, option.text)
                    # list_packaging = np.append(list_packaging, None)
                    list_units = np.append(list_units, 1)
                    list_price = np.append(list_price, price)    

                    list_category_1_name = np.append(list_category_1_name, "Quantity")
                    list_category_1 = np.append(list_category_1, option.text)
                    list_category_2_name = np.append(list_category_2_name, None)
                    list_category_2 = np.append(list_category_2, None)  

            # If neither a table nor a dropdown menu is detected.
            else:
                price = driver.find_element(By.XPATH, "//span[@id='productPrice']").text

                list_search_term = np.append(list_search_term, search_term)
                list_parent_site = np.append(list_parent_site, "msesupplies")
                list_website = np.append(list_website, website_link)
                list_product_name = np.append(list_product_name, product_name)
                list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                # list_quantity = np.append(list_quantity, 1)
                # list_packaging = np.append(list_packaging, None)
                list_units = np.append(list_units, 1)
                list_price = np.append(list_price, price)

                list_category_1_name = np.append(list_category_1_name, None)
                list_category_1 = np.append(list_category_1, None)
                list_category_2_name = np.append(list_category_2_name, None)
                list_category_2 = np.append(list_category_2, None)
        except Exception as e:
            pass  # No data exists on this site. 

In [ ]:
# 3: Sigmaaldrich

def sigmaaldrich(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2

    # Wait for the "reject all" button to be clickable and click it
    try:
        driver.get(website_link) 
        reject_all_button = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept Cookies"]'))
        )
        reject_all_button.click()
    except Exception as e:
        pass
        
    
    if '/product/' in website_link: 
        time.sleep(3)

        # Sigmaaldrich also contains buttons. Howeve, we don't need to select them, but we can identify them. 
        quantities = driver.find_elements(By.XPATH, "//div[@role='button']/span/span")
        prices = driver.find_elements(By.XPATH, "//div[@role='button']/span/div")

        q_mp = len(quantities) // 2
        p_mp = len(prices) // 2

        quantities = quantities[q_mp:]
        prices = prices[p_mp:]

        # Main row-generating scheme
        quantities_and_prices = tuple(zip(quantities, prices))
        for q, p in quantities_and_prices:
            # list_quantity = np.append(list_quantity, q.text)
            list_price = np.append(list_price, p.text)
            list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
            list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S")) # Date - Finished scraping the page

            # list_packaging = np.append(list_packaging, None)
            list_units = np.append(list_units, 1)
            list_parent_site = np.append(list_parent_site, "sigmaaldrich")
            list_website = np.append(list_website, driver.current_url)

            list_search_term = np.append(list_search_term, search_term)

            list_category_1_name = np.append(list_category_1_name, "Quantity")
            list_category_1 = np.append(list_category_1, q.text)
            list_category_2_name = np.append(list_category_2_name, None)
            list_category_2 = np.append(list_category_2, None)

In [ ]:
# 4: Ossila

def ossila(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2
    if 'ossila.com/products' in website_link:
        driver.get(website_link)
        time.sleep(3) # Waiting for the site to load

        # Ossila contains two dropdown menus. We want all combinations of the options in the two dropdown menus - first_object represents the first dropdown menu, second_object for the second. 
        try: 
            first_select_element = driver.find_element(By.XPATH, "//select[@class='single-option-selector' and @data-option='option1']")
            first_object = Select(first_select_element)
            first_options = first_object.options

            for option in first_options:
                first_object.select_by_visible_text(option.text) # Selecting the item

                try:
                    second_select_element = driver.find_element(By.XPATH, "//select[@class='single-option-selector' and @data-option='option2']")
                    second_object = Select(second_select_element)
                    second_options = second_object.options

                    for option2 in second_options:
                        second_object.select_by_visible_text(option2.text)

                        list_search_term = np.append(list_search_term, search_term)
                        list_parent_site = np.append(list_parent_site, "ossila")
                        list_website = np.append(list_website, driver.current_url)
                        list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
                        list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        # list_quantity = np.append(list_quantity, None)
                        # list_packaging = np.append(list_packaging, None)
                        list_price = np.append(list_price, driver.find_element(By.XPATH, "//span[@id='currency-price-span']").text)
                        list_units = np.append(list_units, 1)

                        list_category_1_name = np.append(list_category_1_name, driver.find_element(By.XPATH, "//label[@for='product-select-option-0']").text)
                        list_category_1 = np.append(list_category_1, option.text)
                        list_category_2_name = np.append(list_category_2_name, driver.find_element(By.XPATH, "//label[@for='product-select-option-1']").text)
                        list_category_2 = np.append(list_category_2, option2.text)


                except Exception as e:  # If there's only 1 dropdown
                    list_search_term = np.append(list_search_term, search_term)
                    list_parent_site = np.append(list_parent_site, "ossila")
                    list_website = np.append(list_website, driver.current_url)
                    list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
                    list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    # list_quantity = np.append(list_quantity, None)
                    # list_packaging = np.append(list_packaging, None)
                    list_price = np.append(list_price, driver.find_element(By.XPATH, "//span[@id='currency-price-span']").text)
                    list_units = np.append(list_units, 1)

                    list_category_1_name = np.append(list_category_1_name, driver.find_element(By.XPATH, "//label[@for='product-select-option-0']").text)
                    list_category_1 = np.append(list_category_1, option.text)
                    list_category_2_name = np.append(list_category_2_name, None)
                    list_category_2 = np.append(list_category_2, None)

        except Exception as e:  # If there are no dropdowns. 
            list_search_term = np.append(list_search_term, search_term)
            list_parent_site = np.append(list_parent_site, "ossila")
            list_website = np.append(list_website, driver.current_url)
            list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
            list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            # list_quantity = np.append(list_quantity, 1)
            # list_packaging = np.append(list_packaging, None)
            list_price = np.append(list_price, driver.find_element(By.XPATH, "//span[@id='currency-price-span']").text)
            list_units = np.append(list_units, 1)

            list_category_1_name = np.append(list_category_1_name, None)
            list_category_1 = np.append(list_category_1, None)
            list_category_2_name = np.append(list_category_2_name, None)
            list_category_2 = np.append(list_category_2, None)

    # Additions to be made: perhaps grab the general category: first is typically grade but varies, second is quantity. (Maybe inferable)

In [ ]:
# 5: Tcichemicals

def tcichemicals(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_quantity, list_packaging, list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2
    
    # For tcichemicals, we must open up a new window each time we grab a link to avoid HTTP2 Errors. 
    if 'en/p/' in website_link:
        driver = create_driver()  # To avoid HTTP2 Errors
        try:
            driver.get(website_link)
            WebDriverWait(driver, 10).until(
            lambda d: d.execute_script('return document.readyState') == 'complete')
        except Exception as e:
            try:
                driver.get(website_link)
                WebDriverWait(driver, 10).until(
                lambda d: d.execute_script('return document.readyState') == 'complete')
            except Exception as e:
                pass
                
        # Tcichemicals stores its data in a table. 
        
        sizes = driver.find_elements(By.XPATH, "//table[@id='PricingTable']/tbody/tr/td[@data-attr='Size:']")  # Size
        prices = driver.find_elements(By.XPATH, "//table[@id='PricingTable']/tbody/tr/td[@data-attr='Unit Price']")
        size_price_tuple = tuple(zip(sizes, prices))
        for s, p in size_price_tuple:
            list_search_term = np.append(list_search_term, search_term)
            list_parent_site = np.append(list_parent_site, "tcichemicals")
            list_website = np.append(list_website, driver.current_url)
            list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
            list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            # list_quantity = np.append(list_quantity, s.text)
            # list_packaging = np.append(list_packaging, None)
            list_price = np.append(list_price, p.text)
            list_units = np.append(list_units, 1)

            list_category_1_name = np.append(list_category_1_name, "Quantity")
            list_category_1 = np.append(list_category_1, s.text)
            list_category_2_name = np.append(list_category_2_name, None)
            list_category_2 = np.append(list_category_2, None)

    # Grabbing the size and unit price - table of options

        
    

In [ ]:
''' INCOMPLETE - Ignore this cell
# Chemicalbook
def chemicalbook(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_price, list_units, list_category_1, list_category_1_name, list_category_2, list_category_2_name
    if '.com/Price' in website_link:
        driver.get(website_link)
        time.sleep(2)
        try:
            list_product_name = np.append(list_product_name, driver.find_element(By.TAG_NAME, "h1").text)
            list_search_term = np.append(list_search_term, search_term)
            list_parent_site = np.append(list_parent_site, "chemicalbook")
            list_website = np.append(list_website, driver.current_url)
        
            list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            # list_quantity = np.append(list_quantity, None)
            # list_packaging = np.append(list_packaging, None)

            list_units = np.append(list_units, 1)

            list_category_1_name = np.append(list_category_1_name, None)
            list_category_1 = np.append(list_category_1, None)
            list_category_2_name = np.append(list_category_2_name, None)
            list_category_2 = np.append(list_category_2, None)

            try:
                a = driver.find_element(By.XPATH, "//div[@class='product_base']/ul/li")
                list_price = np.append(list_price, a.text)
            except Exception as e:
                try:
                    a = driver.find_element(By.XPATH, "//div[@class='currency']")
                    list_price = np.append(list_price, a.text)
                except Exception as e:
                    list_price = np.append(list_price, None)
        except Exception as e:
            pass           
'''

In [ ]:
# 6: Thermofisher

def thermofisher(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_price, list_units, list_category_1_name, list_category_1, list_category_2_name, list_category_2
    if 'assets' not in website_link and 'search' not in website_link: # and '/product/' in website_link:
        try:
        # Wait for the "reject all" button to be clickable and click it
            driver.get(website_link) 
            reject_all_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//button[text()="Reject All"]'))
            )
            reject_all_button.click()
        except Exception as e:
            pass   
        
        time.sleep(3)
        
        quantity_buttons = driver.find_elements(By.XPATH, "//button[@data-testid='test-chip-element']/span/span")
        
        if not len(quantity_buttons):  # No buttons on screen

            list_search_term = np.append(list_search_term, search_term)
            list_parent_site = np.append(list_parent_site, "thermofisher")
            list_website = np.append(list_website, driver.current_url)
            
            h1_tags = driver.find_elements(By.TAG_NAME, "h1")
            h1_text = ""
            for h1 in h1_tags:
                if h1.text.strip():
                    h1_text = h1.text
                    break
            list_product_name = np.append(list_product_name, h1_text)
            list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            list_units = np.append(list_units, 1)
            list_category_2_name = np.append(list_category_2_name, None)
            list_category_2 = np.append(list_category_2, None)

            try:
                big_text = driver.find_element(By.XPATH, "//div[@class='ng-binding ng-scope']")
                text_split = re.split(r'[\n\s]+', big_text.text)
                try:
                    list_category_1 = np.append(list_category_1, text_split[3] + " " + text_split[4])
                    list_category_1_name = np.append(list_category_1_name, "Quantity")
                except Exception as e:
                    list_category_1 = np.append(list_category_1, None)
                    list_category_1_name = np.append(list_category_1_name, None)
                try:
                    list_price = np.append(list_price, text_split[5] + " " + text_split[6])
                except Exception as e:
                    list_price = np.append(list_price, None)
            except Exception as e:
                try:
                    table = driver.find_elements(By.XPATH, "//tr[@class='pdp-table__selected-sku']//td")
                    list_category_1 = np.append(list_category_1, table[2].text)
                    list_category_1_name = np.append(list_category_1_name, "Quantity")
                    list_price = np.append(list_price, table[3].text)

                except Exception as e:
                    list_category_1 = np.append(list_category_1, None)
                    list_category_1_name = np.append(list_category_1_name, None)
                    try:
                        list_price = np.append(list_price, driver.find_elements(By.XPATH, "//div[@class='pdp-legal-price']/span")[0].text)
                    except Exception as e:
                        list_price = np.append(list_price, None)

        else:  # If there are buttons on the screen
            for i in quantity_buttons: 
                i.click()
                time.sleep(1)
                actives = driver.find_elements(By.XPATH, "//button[@class='c-chip c-chip--choice c-chip--active']")
                if len(actives) == 0:
                    list_search_term = np.append(list_search_term, search_term)
                    list_parent_site = np.append(list_parent_site, "thermofisher")
                    list_website = np.append(list_website, driver.current_url)
                    
                    h1_tags = driver.find_elements(By.TAG_NAME, "h1")
                    h1_text = ""
                    for h1 in h1_tags:
                        if h1.text.strip():
                            h1_text = h1.text
                            break
                    list_product_name = np.append(list_product_name, h1_text)

                    list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    list_category_1 = np.append(list_category_1, None)
                    list_category_1_name = np.append(list_category_1_name, None)
                    try:
                        list_price = np.append(list_price, driver.find_element(By.XPATH, "//div[@class='pdp-legal-price']/span").text)    
                    except Exception as e:
                        list_price = np.append(list_price, None)
                    list_units = np.append(list_units, 1)

                    list_category_2_name = np.append(list_category_2_name, None)
                    list_category_2 = np.append(list_category_2, None)
                else:
                    list_search_term = np.append(list_search_term, search_term)
                    list_parent_site = np.append(list_parent_site, "thermofisher")
                    list_website = np.append(list_website, driver.current_url)
                    
                    h1_tags = driver.find_elements(By.TAG_NAME, "h1")
                    h1_text = ""
                    for h1 in h1_tags:
                        if h1.text.strip():
                            h1_text = h1.text
                            break
                    list_product_name = np.append(list_product_name, h1_text)

                    list_date = np.append(list_date, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    list_category_1 = np.append(list_category_1, actives[0].text)
                    list_category_1_name = np.append(list_category_1_name, "Quantity")
                    try:
                        list_category_2 = np.append(list_category_2, actives[1].text)
                        list_category_2_name = np.append(list_category_2_name, "Packaging")
                    except Exception as e:
                        list_category_2 = np.append(list_category_2, None)
                        list_category_2_name = np.append(list_category_2_name, None)
                    try:
                        list_price = np.append(list_price, driver.find_element(By.XPATH, "//div[@class='pdp-legal-price']/span").text)
                    except Exception as e:
                        list_price = np.append(list_price, None)
                    list_units = np.append(list_units, 1)
        

In [ ]:
# Helper Functions

# Function: Takes in a website link and search term and runs a function corresponding to one of the 6 sites. 
def web_scrape(website_link, search_term):
    global list_search_term, list_parent_site, list_website, list_product_name, list_date, list_quantity, list_packaging, list_price, list_units

    pattern = re.compile(r'https?://[^.]+\.[^.]+\.')
    reg_link = pattern.findall(website_link)  
    reg_string = reg_link[0]

    if 'fishersci' in reg_string:
        fishersci(website_link, search_term)
    elif 'sigmaaldrich' in reg_string:
        sigmaaldrich(website_link, search_term)
    elif 'msesupplies' in reg_string:
        msesupplies(website_link, search_term)
    elif 'ossila' in reg_string:
        ossila(website_link, search_term)
    elif 'tcichemicals' in reg_string:
        tcichemicals(website_link, search_term)
    # elif 'chemicalbook' in reg_string:
        # chemicalbook(website_link, search_term)
    elif 'thermofisher' in reg_string:
        thermofisher(website_link, search_term)
    else:
        pass # Bad link

# Filtering: Takes in a list (Org_list) and a list of substrings (substrings), filters Org_list for items that are found in substrings. Not explicitly used but can be helpful for running code on certain sites.
def filter_by_substrings(Org_list, substrings):
        return [string for string in Org_list if any(substring in string for substring in substrings)]

# Function that creates a new chromedriver - and opens up a tab. 
def create_driver():
    chrome_options = Options()
    chrome_options.add_argument('--disable-http2')
    driver = webdriver.Chrome(options=chrome_options) 
    return driver

#### General Process

In [ ]:
# Creating the Google search terms: final_search_terms is a list of the terms, as printed out. 

xls = pd.ExcelFile('perovskite_search_terms_altered.xlsx')  # Takes in a excel file. Feel free to add chemicals to a list if necessary. 
search_terms_df = pd.read_excel(xls, "search_terms")
search_terms_list = search_terms_df.iloc[:, 0].to_list()
# Can also hashtag the first three lines of this code, and create a list with chemicals: e.g. search_terms_list = ["NMP"]
search_string = ' chemical product cost'
search_terms_list_with_cost = [i + search_string for i in search_terms_list]
search_string_2 = " site:sigmaaldrich.com OR site:fishersci.com OR site:msesupplies.com OR site:ossila.com OR site:tcichemicals.com OR site:chemicalbook.com OR site:thermofisher.com"
final_search_terms = [j + search_string_2 for j in search_terms_list_with_cost]
print("Number of search terms: " + str(len(final_search_terms)))
final_search_terms

In [ ]:
# Grabbing a list of links to scrape. (By googling our search terms and scraping the links).

links_list = []
search_list = []
key_websites_list = ["sigmaaldrich", "fishersci", "msesupplies", "ossila", "tcichemicals", "chemicalbook", "thermofisher"]

driver = webdriver.Chrome(options = chrome_options) # Start the driver - Opens a Chrome window
for i in final_search_terms:
    driver.get('https://google.com/search?q=' + i)
    web_elements = driver.find_elements(By.XPATH, '//a[@jsname="UWckNb"]')
    links = [web_element.get_attribute('href') for web_element in web_elements]
    filtered_links = filter_by_substrings(links, key_websites_list)
    for j in range(len(filtered_links)):
        search_list = np.append(search_list, i)
    links_list = np.append(links_list, filtered_links)

sites_search_tuple = tuple(zip(links_list, search_list)) # Tuple of links and the search terms that created such links. 

In [ ]:
# Main scraping cell. Expect to take 30-90 minutes, depending on how many chemicals you are extracting, and the speed of your wifi connection. (Example: NREL wifi - takes 33 minutes-1hour for a list of 476+ links.)
driver = create_driver()
for link, search_term in sites_search_tuple:
    web_scrape(link, search_term)

In [ ]:
# Compiling lists into a dataframe.
column_names = ['Search Term', 'Website', 'Parent Site', 'Time Scraped (MDT)', 'Product Name', 'Quantity', 'Packaging', 'Price', 'Units']
df = pd.DataFrame(columns = column_names)
df['Search Term'] = list_search_term
df['Website'] = list_website
df['Parent Site'] = list_parent_site
df['Time Scraped (MDT)'] = list_date  # Change the label (MDT) to the time zone you are in. 

df['Product Name'] = list_product_name
df['Price'] = list_price
df['Units'] = list_units

df["category_1_name"] = list_category_1_name
df["category_1_value"] = list_category_1
df["category_2_name"] = list_category_2_name
df["category_2_value"] = list_category_2

In [ ]:
# Showing the data.
print(df.shape)  # (Number of rows, columns)
df.head()  # First 5 rows of the dataframe

### DataFrame Modifications
* Transforming df into a format ready for analysis.

In [ ]:
# Dropping the "Unnamed" column (if it exists)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
# Removing row duplicates (if any)
df = df.drop_duplicates()
print(df.shape)

In [ ]:
# New column - Chemical Names
df["Chemical Name"] = df["Search Term"].apply(lambda x: x.split('chemical')[0].strip())

In [ ]:
# Units in integer form - not needed
'''
def number_extraction_units(text):
    match = re.search(r'\d+', text)
    number = int(match.group()) if match else 1
    return number
    
df["Units"] = df["Units"].apply(number_extraction_units)
'''

Converting currencies to USD. Conversion rates are in real-time. 

In [ ]:
# Grabbing the numbers from prices - Modifying the current price column into integer form and creating a column containing an indication of currency. 

def number_extraction(text):
    if not text:
        number = None
        rest_of_string = None
    else:
        match = re.search(r'\d+', text)
        number = int(match.group()) if match else None
        rest_of_string = re.sub(r'\d+', '', text).strip()
    return number, rest_of_string

df[["Price", "Currency"]] = df["Price"].apply(lambda x: pd.Series(number_extraction(x)))

In [ ]:
# Setup: Grabbing a list of currency symbols

currency_codes = list_currencies()

currency_symbols = set()

for code in currency_codes:
    try:
        symbol = get_currency_symbol(code, locale='en_US')
        if symbol != '$':
            currency_symbols.add(symbol)
    except:
        pass

currency_symbols_list = sorted(list(currency_symbols))  # A list of all currency symbols, excluding the $ sign\

currency_mapping = {}
for code in currency_codes:
    try:
        symbol = get_currency_symbol(code, locale='en_US')
        if symbol not in currency_mapping:  # Ensure unique symbols
            currency_mapping[symbol] = code
    except:
        pass

symbol_to_currency_code = dict(sorted(currency_mapping.items(), key=lambda item: item[0]))

In [ ]:
# Updating Currency column to contain only the currency symbol and creating a boolean column if the currency is usd. 

def contains(text, items):
    if text is None:
        return 1
    return 0 if any(item in text for item in items) else 1

def raw_currency(text, items):
    if text is None:
        return "$"
    for item in items:
        if item in text:
            return item
    return "$"

df["Currency"] = df["Currency"].apply(lambda x: raw_currency(x, currency_symbols_list))
df["usd"] = df["Currency"].apply(lambda x: contains(x, currency_symbols_list))

In [ ]:
# Using an API to get exchange rates and convert currencies into USD. 

def get_all_exchange_rates(base_currency='USD'):
    url = f"https://api.exchangerate-api.com/v4/latest/{base_currency}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        data = response.json()
        return data['rates']
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return {}

# Get exchange rates for USD
rates = get_all_exchange_rates()

currency_codes = list_currencies()

currency_mapping = {}

for code in currency_codes:
    try:
        symbol = get_currency_symbol(code, locale='en_US')
        if symbol not in currency_mapping:  # Ensure unique symbols
            currency_mapping[symbol] = code
    except:
        pass

def convert_to_usd(price, currency_symbol):
    currency_code = currency_mapping.get(currency_symbol, 'USD')
    if currency_code == 'USD':
        return price
    else:
        rate = rates.get(currency_code, 1)
        return price / rate

df['USD_Price'] = df.apply(lambda row: convert_to_usd(row['Price'], row['Currency']), axis=1)

Standardized Quantity (L, G) form

* Create a quantity column - from the output of the 4 NA ones. 

In [ ]:
# If a row has a quantity value, create a new column with it. 

def quantity_converter(c1, c1_val, c2, c2_val):
    if c1 == "Quantity":
        return c1_val
    elif c2 == "Quantity": 
        return c2_val
    
df["quantity_value"] = df.apply(lambda row: quantity_converter(row["category_1_name"], row["category_1_value"], row["category_2_name"], row["category_2_value"]), axis=1)

In [ ]:
# Extracting a number and units from the quantity.

def quantity_extraction(quantity):
    pattern = re.compile(r'[-0-9Xx]')
    # Formatting issues 
    if not quantity:
        return None, None
    if "X" in quantity:
        numbers = re.findall(r'\d+', quantity)
        num_value = int(math.prod([int(num) for num in numbers]))
        sci_units = pattern.sub('', quantity)
        sci_units = sci_units.replace(" ", "")
        return num_value, sci_units
    elif "-" in quantity: 
        quantity = re.search(r'-(.*)', quantity)[0]
    # Grabbing the units 
    prelim = re.search(r'[0-9]+', quantity)
    if prelim:
        num_value = int(prelim[0])
        sci_units = pattern.sub('', quantity)
        sci_units = sci_units.replace(" ", "")
        return num_value, sci_units
    else:
        return None, None

df[["quantity_number", "quantity_units"]] = df["quantity_value"].apply(lambda x: pd.Series(quantity_extraction(x)))

In [ ]:
# Standardizing the values - converting everything to L or to G
# Account for mL, L, kL, uG, mg, g, kg - would be nice if there was a python package that could account for all 
def standardize(q_number, q_units):
    if "mL" or "ML" or "ml" or "Ml" in q_units:
        return q_number / 1000, "L"
    elif "L" or "l" in q_units:
        return q_number * 1, "L"
    elif "kL" or "KL" or "kl" or "Kl" in q_units:
        return q_number * 1000, "L"
    elif "uG" or "UG" or "ug" or "Ug" in q_units:
        return q_number / 1000000, "G"
    elif "mG" or "MG" or "mg" or "Mg" in q_units:
        return q_number / 1000, "G"
    elif "G" or "g" in q_units:
        return q_number * 1, "G"
    elif "kG" or "KG" or "kg" or "Kg" in q_units:
        return q_number * 1000, "G"
    else:
        return None, None

df[["standardized_units_l_g", "q_units_standardized"]] = df.apply(lambda row: pd.Series(standardize(row["quantity_number"], row["quantity_units"])), axis = 1)

In [ ]:
# Rearranging the column orders
column_order = ["Chemical Name", "Search Term", "Website", "Parent Site", "Time Scraped (MDT)", "Product Name", "Units", "Price", "USD_Price", "Currency", "usd", "category_1_name", "category_1_value", "category_2_name", "category_2_value", "quantity_value", "quantity_number", "quantity_units", "standardized_units_l_g", "q_units_standardized"]
final_df = df[column_order]

In [ ]:
# Final look at the dataframe - columns can be interpreted with help from the README linked above.
print(final_df.shape)
final_df

Importing final dataframe to an Excel file

In [ ]:
# final_df.to_excel("your_name_here.xlsx")
final_df.to_excel("final_web_data.xlsx")